In [68]:
import glob
import PIL
from PIL import Image
import numpy as np
import os
import shutil
from scipy.io import loadmat

# Chemin vers le dossier contenant vos images
image_directory = 'generated_images'

# Créer une liste pour stocker les images chargées
images_list = []
IM_SIZE = 128
# Lire les images depuis le dossier
for image_path in glob.glob(f'{image_directory}/*.jpg'):  # Assurez-vous de changer *.jpg si vos images sont dans un autre format
    with Image.open(image_path) as img:
        img = img.resize((IM_SIZE, IM_SIZE))  # Redimensionne l'image
        img_array = np.asarray(img)  # Convertit l'image en tableau numpy
        images_list.append(img_array)

# Convertit la liste des images en un tableau numpy pour l'entraînement
x_train = np.array(images_list)

print('x_train shape:', x_train.shape)


x_train shape: (1598, 128, 128, 3)


In [69]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, BatchNormalization
from tensorflow.keras.optimizers import Adam
import numpy as np


In [70]:
def build_generator(z_dim):
    model = Sequential()

    # Commencez avec une couche Dense qui prend le bruit d'entrée de dimension z_dim
    model.add(Dense(8 * 8 * 256, input_dim=z_dim))
    model.add(Reshape((8, 8, 256)))

    # Deconvolution, ou Conv2DTranspose, pour agrandir l'image
    model.add(Conv2DTranspose(128, kernel_size=4, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    # Deuxième couche de Conv2DTranspose pour agrandir davantage l'image
    model.add(Conv2DTranspose(64, kernel_size=4, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    
    # Ajout d'une couche de Conv2DTranspose supplémentaire pour agrandir l'image de 64x64 à 128x128

    model.add(Conv2DTranspose(64, kernel_size=4, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # Couche de sortie qui redimensionne l'image à la taille cible
    model.add(Conv2DTranspose(3, kernel_size=4, strides=2, padding='same', activation='tanh'))
     # Ajout d'une couche de Conv2DTranspose supplémentaire pour agrandir l'image de 64x64 à 128x128
   


    return model


In [71]:
def build_discriminator(img_shape):
    model = Sequential()

    model.add(Conv2D(64, kernel_size=4, strides=2, padding='same', input_shape=img_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization())

    model.add(Conv2D(128, kernel_size=4, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization())

    # Flatten the image and output with a Dense layer
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    return model



In [72]:
def build_gan(generator, discriminator):
    model = Sequential()

    # Combiné générateur et discriminateur
    model.add(generator)
    discriminator.trainable = False  # Très important - cela congèle le discriminateur lors de l'entraînement du GAN.
    model.add(discriminator)

    return model


In [73]:
z_dim = 100  # Dimension de l'espace latent (taille du bruit d'entrée)

img_shape = (128, 128, 3)  # Adjust the shape to include the channel dimension
discriminator = build_discriminator(img_shape)
generator = build_generator(z_dim)
build_gan(generator, discriminator)



<Sequential name=sequential_27, built=False>

In [74]:
z_dim = 100  # Dimension de l'espace latent (taille du bruit d'entrée)

# Construisez et compilez le discriminateur
discriminator = build_discriminator((128, 128, 3))
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Construisez le générateur
generator = build_generator(z_dim)

# Construisez et compilez le GAN
gan_model = build_gan(generator, discriminator)
gan_model.compile(loss='binary_crossentropy', optimizer=Adam())


In [79]:
import numpy as np
import matplotlib.pyplot as plt

# Paramètres d'entraînement
epochs = 10000
batch_size = 32
sample_interval = 100  # Intervalle pour sauvegarder des images générées pour le suivi

# Entrée de dimensions pour le générateur (espace latent)
z_dim = 100

# Préparation des étiquettes réelles et fausses
valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))
import numpy as np
import matplotlib.pyplot as plt

# Définition de la fonction sample_images avant de l'utiliser
def sample_images(epoch, generator, z_dim, image_grid_rows=4, image_grid_columns=4):
    # Échantillon de bruit
    noise = np.random.normal(0, 1, (image_grid_rows * image_grid_columns, z_dim))
    gen_imgs = generator.predict(noise)

    # Rééchelonnement des images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    # Configuration de la grille d'images
    fig, axs = plt.subplots(image_grid_rows, image_grid_columns, figsize=(4, 4), sharey=True, sharex=True)

    cnt = 0
    for i in range(image_grid_rows):
        for j in range(image_grid_columns):
            axs[i, j].imshow(gen_imgs[cnt, :, :, :], interpolation='nearest')
            axs[i, j].axis('off')
            cnt += 1
    fig.savefig(f"gen_images/{epoch}.png")
    plt.close()

for epoch in range(epochs):

    # ---------------------
    #  Entraîner le discriminateur
    # ---------------------

    # Sélection aléatoire d'images
    idx = np.random.randint(0, x_train.shape[0], batch_size)
    imgs = x_train[idx]

    # Générer un lot de nouvelles images
    noise = np.random.normal(0, 1, (batch_size, z_dim))
    gen_imgs = generator.predict(noise)

    # Entraîner le discriminateur
    d_loss_real = discriminator.train_on_batch(imgs, valid)
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # ---------------------
    #  Entraîner le générateur
    # ---------------------

    noise = np.random.normal(0, 1, (batch_size, z_dim))

    # Le générateur veut que le discriminateur marque les images générées comme réelles
    g_loss = gan_model.train_on_batch(noise, valid)
    print(f"{epoch} [D loss: {d_loss[0]}, acc.: {100*d_loss[1]}] [G loss: {g_loss}]")   
    # Progression
    if epoch % sample_interval == 0:
        print(f"{epoch} [D loss: {d_loss[0]}, acc.: {100*d_loss[1]}] [G loss: {g_loss}]")

        # Sauvegarde des images générées pour suivi
        sample_images(epoch, generator, z_dim)




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
0 [D loss: 1.787093162536621, acc.: 17.669808864593506] [G loss: 0.5310404300689697]
0 [D loss: 1.787093162536621, acc.: 17.669808864593506] [G loss: 0.5310404300689697]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1 [D loss: 1.7942100763320923, acc.: 17.24959909915924] [G loss: 0.5173099040985107]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
2 [D loss: 1.8180150985717773, acc.: 17.24647879600525] [G loss: 0.5041614770889282]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
3 [D loss: 1.830306887626648, acc.: 17.099960148334503] [G loss: 0.4911381006240845]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
4 [D loss: 1.8183693885803223, acc.: 17.44716167449951] [G loss: 0.47900131344795227]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
5 [D loss: 1.824104905128479, acc.: 17.501938343048096] [G loss: 0.4677799344062805]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
6 [D loss: 1.8324271440505981, acc.: 17.552295327186584] [G loss: 0.45707690715789795]
1/1 ━━━━━━━━━

: 